## 1.0 Importing Packages

#### 1.1 Import standard packages

In [4]:
import numpy as np
import pandas as pd
import matplotlib as mpl
mpl.use('agg')
import matplotlib.pyplot as plt
import time
from emcee import EnsembleSampler
import corner

#### 1.2 Install + Import GALARIO

In [2]:
conda install -c conda-forge galario

Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /srv/conda/envs/notebook

  added / updated specs:
    - galario


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    fftw-3.3.10                |nompi_hc118613_108         1.9 MB  conda-forge
    galario-1.2.2              |py38h7191ed2_1003         315 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  fftw               conda-forge/linux-64::fftw-3.3.10-nompi_hc118613_108 
  galario            conda-forge/linux-64::galario-1.2.2-py38h7191ed2_1003 



fftw-3.3.10          | 1.9 MB    |                                       |   0% 
fftw-3.3.10          | 1.9 MB    | 3                                     |   1% 
galario-1.2

In [3]:
import galario.double as g_double
from galario import arcsec, deg

## 2.0 Define Functions

In [33]:
def lnpriorfn(p, par_ranges):

    """ Uniform prior probability function """

    for i in range(len(p)):
        if p[i] < par_ranges[i][0] or p[i] > par_ranges[i][1]:
            return -np.inf

    jacob = -p[0]

    return jacob

In [34]:
def lnpostfn(p, p_ranges, rmin, dr, nr, nxy, dxy, u, v, re, im, w):

    """ Log of posterior probability function """

    ### apply prior
    lnprior = lnpriorfn(p, p_ranges)
    if not np.isfinite(lnprior):
        return -np.inf

    ### unpack the parameters
    if args_r_in:
        f0, sigma, r_in, inc, pa, dra, ddec = p
        r_in *= arcsec
    else:
        f0, sigma, inc, pa, dra, ddec = p

    ### convert to correct units
    f0 = 10.**f0
    sigma *= arcsec
    inc *= deg
    pa *= deg
    dra *= arcsec
    ddec *= arcsec

    ### get gaussian profile
    f, r = GaussianProfile(f0, sigma, rmin, dr, nr)

    ### add inner cavity?
    if args_r_in:
        f[r < r_in] *= 0

    ### calculate chi-squared
    chi2 = g_double.chi2Profile(f, rmin, dr, nxy, dxy, u, v, re, im, w,
                                inc=inc, PA=pa, dRA=dra, dDec=ddec)

    ### return likelihood
    return -0.5 * chi2 + lnprior

In [35]:
def GaussianProfile(f0, sigma, rmin, dr, nr):

    """ Gaussian brightness profile. """

    ### calculate radial grid
    r = np.linspace(rmin, rmin + dr * nr, nr, endpoint=False)

    ### calculate gaussian profile
    f = f0 * np.exp(-0.5 * (r / sigma)**2)

    ### return gaussian profile
    return f, r

## 3.0 Main Code

In [21]:
### setup args
args_nsteps = 3000
args_nthreads = 12
args_wavelength = 1.33e-3
args_f0, args_sigma = 10.0, 0.2
args_incl, args_pa = 65.0, 22.0
args_dra, args_ddec = 0.09, -0.44
args_r_in = 0.0
args_uvtable = "uvtable_EPIC_203794605.txt"
args_restart = ""

### print to screen
print(f"\nInputs:\n\tIncl = {args_incl} deg\n\tPA = {args_pa} deg\n\tR_in = {args_r_in} AU\n")


Inputs:
	Incl = 65.0 deg
	PA = 22.0 deg
	R_in = 0.0 AU



In [22]:
### get initial guesses and ranges of gaussian fit parameters
if args_r_in != 0.0:
    p0 = [args_f0, args_sigma, args_r_in, args_incl, args_pa, args_dra, args_ddec]
    p_ranges = [[0.1, 100.0], [0.01, 5.0], [0.01, 5.0], [0., 90.], [0., 180.], [-2.0, 2.0], [-2.0, 2.0]]
else:
    p0 = [args_f0, args_sigma, args_incl, args_pa, args_dra, args_ddec]
    p_ranges = [[1.0, 50.0], [0.01, 5.0], [0., 90.], [0., 180.], [-2.5, 2.5], [-2.5, 2.5]]

In [23]:
### setup mcmc
ndim = len(p0)
nwalkers = ndim * 20 
nsteps = int(args_nsteps / 5)
tsteps = args_nsteps
nthreads = args_nthreads
print(f"\nEmcee setup:\n\tSteps = {tsteps}\n\tWalkers = {nwalkers}\n\tThreads = {nthreads}\n")


Emcee setup:
	Steps = 3000
	Walkers = 120
	Threads = 12



In [24]:
### read in UV table visibilities
print("\nReading in UV table: " + args_uvtable)
U, V, Re, Im, W = np.loadtxt(args_uvtable, unpack=True)
U, V = np.ascontiguousarray(U), np.ascontiguousarray(V)
U /= args_wavelength
V /= args_wavelength


Reading in UV table: uvtable_EPIC_203794605.txt


In [25]:
### get image dimensions
print("\nImage size: ")
Nxy, Dxy = g_double.get_image_size(U, V, verbose=True, f_max=2.1, f_min=2.0)
Rmin, dR, nR = 1e-4 * arcsec, 0.0005 * arcsec, 20000

### get initial guesses and ranges of gaussian fit parameters
if args_r_in != 0.0:
    print(f"\nUsing inner hole = {args_r_in} AU")
    p0 = [args_f0, args_sigma, args_r_in, args_incl, args_pa, args_dra, args_ddec]
    p_ranges = [[0.1, 100.0], [0.01, 5.0], [0.01, 5.0], [0., 90.], [0., 180.], [-2.0, 2.0], [-2.0, 2.0]]
else:
    p0 = [args_f0, args_sigma, args_incl, args_pa, args_dra, args_ddec]
    p_ranges = [[1.0, 50.0], [0.01, 5.0], [0., 90.], [0., 180.], [-2.5, 2.5], [-2.5, 2.5]]


Image size: 
dxy:2.136993e-02arcsec	nxy_MRS:1024
nxy_MRS: matrix size to have FOV > f_min * MRS, where f_min:2.0 and MRS:1.094140e+01arcsec


In [36]:
### set sampler and initial positions of walkers
sampler = EnsembleSampler(nwalkers, ndim, lnpostfn, args=[p_ranges, Rmin, dR, nR, Nxy, Dxy, U, V, Re, Im, W], threads=nthreads)
if args_restart != "":
    pos = np.load(args_restart)[:, -1, :]
    print("Restarting from " + args_restart)
else:
    pos = [p0 + 1e-4 * np.random.randn(ndim) for i in range(nwalkers)]

In [37]:
### set output directory
outdir = '.'

### set labels for plotting
if args_r_in != 0.0:
    labels = [r"$f_0$", r"$\sigma$", r"$r_{min}$", r"$incl$", r"PA", r"$\Delta$RA", r"$\Delta$Dec"]
else:
    labels = [r"$f_0$", r"$\sigma$", r"$incl$", r"PA", r"$\Delta$RA", r"$\Delta$Dec"]

In [38]:
### do mcmc fit
print("\nStarting fit...\n")
start = time.time()
prob, state = None, None
for j in range(nsteps, tsteps + 1, nsteps):

    ### get last 500 samples
    pos, prob, state = sampler.run_mcmc(pos, nsteps, rstate0=state, lnprob0=prob)
    samples = sampler.chain[:, -500:, :].reshape((-1, ndim))

    ### plot corner plot ever nsteps
    fig = corner.corner(samples, labels=labels, show_titles=True, quantiles=[0.16, 0.50, 0.84], label_kwargs={'labelpad': 20, 'fontsize': 0}, fontsize=8)
    fig.savefig(os.path.join(outdir, "corner_{}.png".format(j)))
    plt.close('all')

    ### output walkers every nsteps
    np.save(os.path.join(outdir, "chain_{}".format(j)), sampler.chain)
    print("{0} steps completed: chain saved in chain_{0}.npy - corner plot saved in triangle_{0}".format(j))

emcee: Exception while calling your likelihood function:
  params: [10.00002309  0.20000146 64.99987871 21.99997327  0.09021526 -0.44009737]
  args: [[[1.0, 50.0], [0.01, 5.0], [0.0, 90.0], [0.0, 180.0], [-2.5, 2.5], [-2.5, 2.5]], 4.84813681109536e-10, 2.42406840554768e-09, 20000, 1024, 1.0360434721060641e-07, array([  15891.03759398,  114639.54887218,  185106.16541353, ...,
       -282530.22556391, -455810.52631579,  173280.37593985]), array([  80163.30827068,   30404.2556391 ,  336781.65413534, ...,
        158706.69172932, -523256.54135338,  681963.23308271]), array([ 0.00545857,  0.01371012, -0.01246594, ..., -0.02020986,
        0.00566444, -0.00616318]), array([-0.00567272,  0.00058593,  0.01321936, ...,  0.00849323,
       -0.01128046,  0.00274976]), array([20469.96, 20108.38, 18221.36, ..., 20374.04, 19066.06, 20623.62])]
  kwargs: {}
  exception:


Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/emcee/ensemble.py", line 624, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "/tmp/ipykernel_211/850060284.py", line 33, in lnpostfn
    chi2 = g_double.chi2Profile(f, rmin, dr, nxy, dxy, u, v, re, im, w,
  File "libcommon.pyx", line 630, in libcommon.chi2Profile
  File "stringsource", line 658, in View.MemoryView.memoryview_cwrapper
  File "stringsource", line 349, in View.MemoryView.memoryview.__cinit__
ValueError: ndarray is not C-contiguous


ValueError: ndarray is not C-contiguous